 ## Income statement analysis tool
 
 
Tools:

Alpha Vantage:
Alpha Vantage allows API calls for free on data such as balance sheet, income statement, real time quotes and more. 
We'll be using it for our API Calls. 
  

API Key: 385ZUF7ZLAH3LO8V


In [91]:
import requests
from pandas import DataFrame



def get_data(stock_index):
    
    # Gets the data and stores it in a dict
    url = 'https://www.alphavantage.co/query?function=INCOME_STATEMENT&symbol=' + str(stock_index) + '&apikey=385ZUF7ZLAH3LO8V'
    r = requests.get(url)
    data = r.json()
    return data



def what_appropriate_unit(number):
    
# Input a number (Gross profit for eg.) Returns the appropriate unit for the number. 
# Will only be used once to set the whole table to the right unit 
# To get access to coef: what_appropriate_unit(number)[0]  - Change to 1 for unit

    if number > 10000000000 or number < -10000000000:
        coef = 10**-9
        unit = 'billon'
    else:
         coef = 10**-6
         unit = 'million'
    return coef, unit   




In [92]:
stock_index = input('what stock do you want to analyze?')
data = get_data(stock_index)
annualReports = data['annualReports']
# Gets us the income statement for the last 5 Years 

#df1 =  DataFrame1, which is unfiltered
df1 =  DataFrame(annualReports)
currency = annualReports[0]['reportedCurrency']
df1 = df1.transpose()
DatesOfIS = df1.values.tolist()[0][0:len(annualReports)]
df1.drop(df1.index[[0, 1]], inplace = True)

if annualReports[0]['researchAndDevelopment'] == 'None':
    df1.drop(df1.index[6], inplace = True)


# Now we have a slightly cleaner version of the income statement without R&D, date and currency (both stored)
# Lets get the units a lil cleaner: Itterate and use new values in million or billion
# Some value have 'None', we need to get rid of the 'None' to convert to int and multiply by coef


df1.replace('None', 0, inplace = True)    

df1 = df1.astype('int64')

coef = what_appropriate_unit(int(df1[0]['totalRevenue']))[0]
unit = what_appropriate_unit(int(df1[0]['totalRevenue']))[1]

df1 = df1.mul(coef)
df1 = df1.round(1)

print(f'All numbers in {unit} {currency}')

All numbers in billon USD


In [93]:
df1

,0,1,2,3,4
grossProfit,105.0,98.4,101.8,88.2,84.3
totalRevenue,271.6,256.6,265.6,229.2,215.6
costOfRevenue,189.5,180.0,163.8,141.0,131.4
costofGoodsAndServicesSold,169.6,161.8,163.8,141.0,131.4
operatingIncome,66.3,63.9,70.9,61.3,60.0
sellingGeneralAndAdministrative,19.9,18.2,16.7,15.3,14.2
researchAndDevelopment,18.8,16.2,14.2,11.6,10.0
operatingExpenses,38.7,34.5,30.9,26.8,24.2
investmentIncomeNet,3.8,5.0,5.7,5.2,4.0
netInterestIncome,-2.9,-3.6,-3.2,-2.3,-1.5


In [94]:
# adding stdev but only for the latest value

list_stdev = []
for index_of_row in range(df1.shape[0]):
    list_stdev.append(round(df1.iloc[index_of_row].std(), 1))

number_of_columns = len(df1.columns)
df1.insert(number_of_columns, 'Stdev', list_stdev)

df1

,0,1,2,3,4,Stdev
grossProfit,105.0,98.4,101.8,88.2,84.3,8.9
totalRevenue,271.6,256.6,265.6,229.2,215.6,24.2
costOfRevenue,189.5,180.0,163.8,141.0,131.4,24.8
costofGoodsAndServicesSold,169.6,161.8,163.8,141.0,131.4,16.4
operatingIncome,66.3,63.9,70.9,61.3,60.0,4.3
sellingGeneralAndAdministrative,19.9,18.2,16.7,15.3,14.2,2.3
researchAndDevelopment,18.8,16.2,14.2,11.6,10.0,3.5
operatingExpenses,38.7,34.5,30.9,26.8,24.2,5.8
investmentIncomeNet,3.8,5.0,5.7,5.2,4.0,0.8
netInterestIncome,-2.9,-3.6,-3.2,-2.3,-1.5,0.8


In [95]:
# Change the value of the header with dates of fiscalDateEnding

for a in range(len(DatesOfIS)):
    df1.rename(columns={a: str(DatesOfIS[a])}, inplace=True)

df1

,2020-09-30,2019-09-30,2018-09-30,2017-09-30,2016-09-30,Stdev
grossProfit,105.0,98.4,101.8,88.2,84.3,8.9
totalRevenue,271.6,256.6,265.6,229.2,215.6,24.2
costOfRevenue,189.5,180.0,163.8,141.0,131.4,24.8
costofGoodsAndServicesSold,169.6,161.8,163.8,141.0,131.4,16.4
operatingIncome,66.3,63.9,70.9,61.3,60.0,4.3
sellingGeneralAndAdministrative,19.9,18.2,16.7,15.3,14.2,2.3
researchAndDevelopment,18.8,16.2,14.2,11.6,10.0,3.5
operatingExpenses,38.7,34.5,30.9,26.8,24.2,5.8
investmentIncomeNet,3.8,5.0,5.7,5.2,4.0,0.8
netInterestIncome,-2.9,-3.6,-3.2,-2.3,-1.5,0.8


In [96]:
# adding growth rates to the first column

# Verifying that the company has reported more that once


if len(DatesOfIS) == 1: 
    pass
else:
    growth_rates = []
    for index_of_row in range(df1.shape[0]):
        growth_rate = ((df1.iloc[index_of_row][0]/df1.iloc[index_of_row][1])-1)*100
        growth_rates.append(round(growth_rate, 1))
    print(growth_rates)

df1.insert(1, 'GRates this year (in %)', growth_rates)

df1

[6.7, 5.8, 5.3, 4.8, 3.8, 9.3, 16.0, 12.2, -24.0, -19.4, -24.0, -19.4, 5.5, -125.0, -14.2, -11.2, 2.1, -7.6, -19.4, 3.8, -0.9, 1.0, -1.1, 3.8]


,2020-09-30,GRates this year (in %),2019-09-30,2018-09-30,2017-09-30,2016-09-30,Stdev
grossProfit,105.0,6.7,98.4,101.8,88.2,84.3,8.9
totalRevenue,271.6,5.8,256.6,265.6,229.2,215.6,24.2
costOfRevenue,189.5,5.3,180.0,163.8,141.0,131.4,24.8
costofGoodsAndServicesSold,169.6,4.8,161.8,163.8,141.0,131.4,16.4
operatingIncome,66.3,3.8,63.9,70.9,61.3,60.0,4.3
sellingGeneralAndAdministrative,19.9,9.3,18.2,16.7,15.3,14.2,2.3
researchAndDevelopment,18.8,16.0,16.2,14.2,11.6,10.0,3.5
operatingExpenses,38.7,12.2,34.5,30.9,26.8,24.2,5.8
investmentIncomeNet,3.8,-24.0,5.0,5.7,5.2,4.0,0.8
netInterestIncome,-2.9,-19.4,-3.6,-3.2,-2.3,-1.5,0.8
